In [37]:
from ogb.nodeproppred import DglNodePropPredDataset
from dgl.dataloading.neighbor import MultiLayerNeighborSampler
from dgl.dataloading.pytorch import NodeDataLoader
from dgl.nn import GATConv

In [8]:
d = DglNodePropPredDataset(name='ogbn-products')

In [9]:
g, labels = d[0]
g, labels

(Graph(num_nodes=2449029, num_edges=123718280,
       ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32)}
       edata_schemes={}),
 tensor([[0],
         [1],
         [2],
         ...,
         [8],
         [2],
         [4]]))

In [17]:
g.ndata['feat'].shape

torch.Size([2449029, 100])

In [11]:
g = g.to("cuda:0")

In [16]:
d.get_idx_split()

{'train': tensor([     0,      1,      2,  ..., 196612, 196613, 196614]),
 'valid': tensor([196615, 196616, 196617,  ..., 235935, 235936, 235937]),
 'test': tensor([ 235938,  235939,  235940,  ..., 2449026, 2449027, 2449028])}

In [18]:
g.ndata['feat'].device

device(type='cuda', index=0)

In [20]:
labels.squeeze(1)

tensor([0, 1, 2,  ..., 8, 2, 4])

In [10]:
sampler = MultiLayerNeighborSampler([15, 15, 15])
split_idx = d.get_idx_split()
train_idx = split_idx["train"]#.to(device)
valid_idx = split_idx["valid"]#.to(device)
test_idx = split_idx["test"]#.to(device)
train_dataloader = NodeDataLoader(g,
                                  train_idx,
                                  sampler,
                                  batch_size=1024,
                                  shuffle=True,
                                  drop_last=False,
#                                       num_workers=4,
                                  )    

In [56]:
input_nodes, seeds, blocks = next(iter(train_dataloader))

In [15]:
input_nodes.shape

torch.Size([871532])

In [61]:
seeds

tensor([ 62152,  24565, 118068,  ..., 128546,  25811, 181088])

In [57]:
blocks

[Block(num_src_nodes=880459, num_dst_nodes=176257, num_edges=2582594),
 Block(num_src_nodes=176257, num_dst_nodes=15676, num_edges=230989),
 Block(num_src_nodes=15676, num_dst_nodes=1024, num_edges=15050)]

In [58]:
blocks[0].number_of_dst_nodes()

176257

In [59]:
blocks[1].ndata['feat']['_N'].shape

torch.Size([176257, 100])

In [54]:
blocks[0].dstdata['feat'][0], blocks[0].srcdata['feat'][0]

(tensor([ 0.5732, -0.3104,  0.3387,  1.4321, -0.3140,  1.0019, -0.4314,  0.0523,
         -0.9290,  0.2076, -0.2914,  1.1955,  0.5039,  0.6504,  0.0198,  0.3723,
         -0.4141,  0.6265, -0.2357,  0.0458,  0.0891, -0.2032, -0.0938, -0.5144,
          0.3038, -0.6644, -0.6155,  0.2167,  0.6638,  0.7903, -0.2806, -0.2437,
         -0.4158,  1.0209, -0.3888,  0.7717, -0.1969, -0.5606,  0.3234,  0.6431,
          0.6773, -0.3765, -0.5819,  0.3278, -1.2331,  0.5929,  0.4997,  0.5553,
          0.3544, -0.8994,  0.0871,  2.0412,  1.9411, -0.4537, -0.0190, -1.5940,
          1.0367,  0.1635, -1.1756, -0.3969,  0.9795, -0.3577,  1.0506,  0.9920,
         -1.2444, -0.1570,  0.8460, -0.1106,  0.5636, -1.5201,  1.7386,  1.2931,
         -0.8524,  0.8753, -0.6568,  0.8314,  0.2860,  2.6468, -0.6079, -0.4724,
          0.6979, -1.0838, -2.3402, -0.2772, -1.2829, -0.1060, -1.6026, -0.6662,
          0.9065, -0.0234, -1.1551,  0.8833, -0.2914, -0.0225,  0.3380,  2.1940,
          0.9189, -0.5383, -

In [60]:
blocks[0].dstnodes(), blocks[0].srcnodes()

(tensor([     0,      1,      2,  ..., 176254, 176255, 176256]),
 tensor([     0,      1,      2,  ..., 880456, 880457, 880458]))

In [45]:
gat = GATConv(100,
        64,
        num_heads=3,
        residual=True,
        allow_zero_in_degree=True)
gat

GATConv(
  (fc): Linear(in_features=100, out_features=192, bias=False)
  (feat_drop): Dropout(p=0.0, inplace=False)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (res_fc): Linear(in_features=100, out_features=192, bias=False)
)

In [44]:
rst = gat(blocks[0], blocks[0].srcdata['feat'])

RuntimeError: The size of tensor a (176010) must match the size of tensor b (871532) at non-singleton dimension 0

In [42]:
rst.shape

torch.Size([176010, 3, 64])

In [20]:
blocks[0].__dir__()

['_graph',
 '_canonical_etypes',
 '_batch_num_nodes',
 '_batch_num_edges',
 '_ntypes',
 '_srctypes_invmap',
 '_dsttypes_invmap',
 '_is_unibipartite',
 '_etypes',
 '_etype2canonical',
 '_etypes_invmap',
 '_node_frames',
 '_edge_frames',
 '__module__',
 '__doc__',
 'is_block',
 '__repr__',
 '__init__',
 '_init',
 '__setstate__',
 '__copy__',
 'add_nodes',
 'add_edge',
 'add_edges',
 'remove_edges',
 'remove_nodes',
 '_reset_cached_info',
 'is_unibipartite',
 'ntypes',
 'etypes',
 'canonical_etypes',
 'srctypes',
 'dsttypes',
 'metagraph',
 'to_canonical_etype',
 'get_ntype_id',
 'get_ntype_id_from_src',
 'get_ntype_id_from_dst',
 'get_etype_id',
 'batch_size',
 'batch_num_nodes',
 'set_batch_num_nodes',
 'batch_num_edges',
 'set_batch_num_edges',
 'nodes',
 'srcnodes',
 'dstnodes',
 'ndata',
 'srcdata',
 'dstdata',
 'edges',
 'edata',
 '_find_etypes',
 '__getitem__',
 'number_of_nodes',
 'num_nodes',
 'number_of_src_nodes',
 'num_src_nodes',
 'number_of_dst_nodes',
 'num_dst_nodes',
 'nu